In [ ]:
# import NumPy for linear algebra computations
import numpy as np
from numpy.linalg import matrix_power 

# import random for generating random numbers
import random

# import copy for copying lists
import copy

In [ ]:
# mathematical constant
SQRT_2 = np.sqrt(2)

# single qubit gates
I = np.eye(2)
H = (1/SQRT_2) * np.array([[1,1],[1,-1]])
Z = np.array([[1,0],[0,-1]])
X = np.array([[0,1],[1,0]])
P = np.array([[1,0],[0,1j]])
T = np.array([[1,0],[0, np.exp(1j * np.pi/4)]])

# two-qubit gates
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
SWAP = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])

In [ ]:
def gen_qubit(alpha, beta):
    """
    Generate a single qubit according to the probability amplitudes alpha and beta 
    and the constraint alpha^2 + beta^2 = 1.
    
    Args: 
        - alpha: a real number between 0 and 1 inclusive
        - beta: a real number between 0 and 1 inclusive
        
    Returns: 
        - the vector alpha |0> + beta |1>
    """
    return np.array([[alpha],[beta]]) # alpha^2 + beta^2 = 1

def zero_state():
    """
    Generate the zero state |0>.
    
    Returns:
        - the vector 1|0>
    """
    return gen_qubit(1,0)


def one_state():
    """
    Generate the one state |1>.
    
    Returns:
        - the vector 1|1>
    """
    
    return gen_qubit(0,1)


def superposition():
    """
    Generate the plus state |+> = 1/sqrt(2) * (|0> + |1>).
    
    Returns:
        - the vector 1/sqrt(2) * (|0> + |1>)
    """
    kat_zero = zero_state()
    return np.matmul(H, kat_zero)


def gen_epr():
    """
    Generate the EPR state 1/sqrt(2) * (|00> + |11>).
    
    Returns:
        - the vector 1/sqrt(2) * (|00> + |11>)
    """
    # generate two quantum registers and initialize them to the zero state
    qubit_1, qubit_2 = zero_state(), zero_state()
    
    # apply the H gate to the first qubit
    qubit_1 = np.matmul(H, qubit_1)
    
    # apply the CNOT gate and obtain the EPR state
    epr = np.matmul(CNOT, np.kron(qubit_1, qubit_2))
    
    return epr

def pure_to_density(psi):
    """
    Compute the density matrix of a pure state |\psi>.
    
    Args: 
        - psi: the vector describing a pure state.
        
    Returns:
        - |\psi> <\psi|.
    """
    return np.matmul(psi, psi.conj().T)


def computational_basis(n: int):
    """
    Generate all the basis elements of an n-dimension qubit system in the computational basis
    For example, if n=1, the function will return the set {|0>,|1>} as a numpy array.
    
    Args:
        - n: the dimension of the system
        
    Returns:
        - arrays: all the basis elements of the n-dimensional qubit system in the computational basis  
    """
    # create 2^n zero arrays
    arrays = [np.zeros((2**n, 1)) for i in range(2**n)]
    
    # substitute 1 for the i^th element of the i^th array
    for i in range(2**n):
        arrays[i][i] = 1
        
    return arrays

def measurement(rho): # measurement is performed in the computational basis
    """
    Return a random string of bits upon the measurement of the density matrix \rho
    according to projections onto the computational basis. 
    
    Args:
        rho: density matrix of a quantum state.
        
    Returns:
        - classical_bits: a string of bits with the same length as the dimension of the quantum state \rho.
    """
    # compute the dimension of the system
    dimension = int(np.log2(rho.shape[0]))
    
    # generate all the basis elements
    computational_bases = computational_basis(dimension)
    
    # initialize a list with 2^n elements with zero
    # which stores the probability amplitudes that 
    # result from the projections onto the computational basis
    probabilities = [0]*(2**dimension)
    
    for i in range(2**dimension):
        probabilities[i] = np.abs(np.trace(np.matmul(rho, pure_to_density(computational_bases[i]))))
    
    # randomly make a choice based on the probability amplitudes
    measurement_res = np.random.choice(len(probabilities), p = probabilities)
    
    classical_bits = bin(measurement_res)[2:].zfill(dimension)
    return classical_bits


def operator_builder(string, L):
    """
    Concatenate a string of "I"s whose positions are specified by the list L with the string variable.
    For example, operator_builder("01", [0,2]) returns the string "I0I1".
    This function will be useful for computing the partial trace of a quantum state.

    Args: 
        - string: a binary string
        - L: a list
        
    Returns: 
        - the string "I" at positions specified by the list L concatenated with the string
    """
    counter = 0
    
    # total length of the string
    n = len(string) + len(L)
    
    # initalize a 0 string as a list
    new_string = list("0"*n)
    
    # set the L[i] location of the new string to "I"
    for i in range(len(L)):
        new_string[L[i]] = "I"
        
    # set the remaining positions according to the string variable that was passed by as an argument
    for j in range(n):
        if new_string[j] == "I":
            pass
        else:
            new_string[j] = string[counter]
            counter = counter + 1
    
    # turn the list into a string variable
    return "".join(new_string)


def partial_trace(rho, L):
    """
    Compute the partial trace of a density matrix rho, while keeping the registers specified in the list L
    and tracing out registers that are not in L.
    
    Args:
        - rho: density matrix of a quantum state
        - L: registers to keep
        
    Returns: 
        - rho_density: partial trace of rho
    """
    system_dimension = int(np.log2(rho.shape[0]))
    trace_out_dimension = system_dimension - len(L)    
    rho_density = np.zeros((2**len(L), 2**len(L)))
    
    for i in range(2**trace_out_dimension):
        operator_matrix = np.array(1)
        operator = bin(i)[2:].zfill(trace_out_dimension)
        operator = operator_builder(operator, L)

        for j in operator:
            if j == "0":
                operator_matrix = np.kron(operator_matrix, zero_state())
            if j == "1":
                operator_matrix = np.kron(operator_matrix, one_state())
            if j == "I":
                operator_matrix = np.kron(operator_matrix, I)   

        rho_density = rho_density + np.matmul(np.matmul(operator_matrix.conj().T, rho), operator_matrix)
        
    return rho_density

def qgate_on_density(quantum_gate, rho):
    """
    Applies a quantum gate on density matrix.
    
    Args: 
        - quantum_gate: a numpy array as the matrix representation of a quantum gate
        - rho: density matrix of a quantum state
        
    Returns:
        - quantum gate * rho * tranpose(conjugate(quantum gate))
    """
    return np.matmul(np.matmul(quantum_gate, rho), quantum_gate.conj().T)

def post_measurement_state(rho, L, bits): 
    """
    Given a density matrix rho and the results of a measurement of a sub-system of rho, 
    return the post-measurement state.
    
    Args: 
        - rho: density matrix of a quantum state
        - L: indices of sub-system(s) of rho that are measured as a list
        - bits: a binary string indicating the results of the measured sub-system(s)
        
    Returns:
        - rho_pms: the density matrix of the post-measurement state
        - bits: the same as the input of the matrix
    """
    system_dimension = int(np.log2(rho.shape[0]))
    operator = list("I"*system_dimension)
    for i in range(len(L)):
        operator[L[i]] = bits[i]
    operator = ''.join(i for i in operator)
    projection_operator = np.array(1)
    zero_projection = np.kron(zero_state(), zero_state().conj().T)
    one_projection = np.kron(one_state(), one_state().conj().T)
    
    for i in operator:
        if i == "0":
            projection_operator = np.kron(projection_operator, zero_projection)
            
        if i == "1":
            projection_operator = np.kron(projection_operator, one_projection)
            
        if i == "I":
            projection_operator = np.kron(projection_operator, I)
    
    rho_pms = (projection_operator @ rho @ projection_operator.conj().T) / (np.trace(projection_operator.conj().T @ projection_operator @ rho))
 
    return (rho_pms, bits)

def qotp_enc(psi, a, b):
    """
    Encrypt the pure quantum state |psi> with a list of bits a and b using quantum one-time pad
    
    Args: 
        - psi: a vector representation of the quantum state |psi>
        - a: a list of bits needed for the X-Pauli gates of the quantum one-time pad
        - b: a list of bits needed for the Z-Pauli gates of the quantum one-time pad
        
    Returns:
        - psi_enc: vector representation of the encrypted state
    """
    xz_tensor = np.matmul(matrix_power(X, a[0]),  matrix_power(Z, b[0]))
    n = len(a)
    for i in range(1,n):
        xz_tensor = np.kron(xz_tensor, np.matmul(np.linalg.matrix_power(X, a[i]),  
                                                 np.linalg.matrix_power(Z, b[i])))
    
    psi_enc = np.matmul(xz_tensor, psi)
    return psi_enc


def qotp_dec_density(enc_rho, a,b):
    """
    Decrypt the density matrix rho with a list of bits a and b using quantum one-time pad.
    
    Args: 
        - enc_rho: a density matrix representation of the encrypted quantum state rho
        - a: a list of bits needed for the X-Pauli gates of the quantum one-time pad
        - b: a list of bits needed for the Z-Pauli gates of the quantum one-time pad
        
    Returns:
        - dec_rho: density matrix of the decrypted state
    """
    xz_tensor = np.matmul(np.linalg.matrix_power(Z, b[0]), np.linalg.matrix_power(X, a[0])) 
    n = len(a)
    for i in range(1,n):
        xz_tensor = np.kron(xz_tensor, np.matmul(np.linalg.matrix_power(Z, b[i]), 
                                                 np.linalg.matrix_power(X, a[i])))
        
    dec_rho = qgate_on_density(xz_tensor, enc_rho)
    return dec_rho

def tcl_layers(circuit):
    """
    Seperate T + Clifford gates of each layer of a quantum circuit. 
    
    Args:
        - circuit: a quantum circuit as a list of strings. each member of the list
        represents a layer in the quantum circuit.
        
    Returns:
        - layered_circuit: a nested list where each nested list seperates individual gates. 
    """
    qgates = ["CNOT", "Z", "I", "X", "P", "H", "T"]
    layered_circuit = []
    
    length_current_layer = 0 
    for layer in range(len(circuit)): 
        length_current_layer = len(circuit[layer])
        counter = 0
        current_gates = []
        for gates in range(length_current_layer):
            if counter >= length_current_layer:
                break
            if circuit[layer][counter] != "C":
                current_gates.append(circuit[layer][counter])
                counter += 1
            else:
                current_gates.append("CNOT")
                counter += 4
        layered_circuit.append(current_gates)
        
    return layered_circuit


def cl_eval_no_fhe(circuits, enc_psi):
    """
    Perform clifford circuit on the encrypted pure state homomorphically.
    For more information refer to the Clifford scheme by Broadbent and Jeffery: https://arxiv.org/abs/1412.8766
    
    Args: 
        - circuits: description of the Clifford circuit as a list of strings
        - enc_psi: a vector representation of the encrypted pure state |psi>_{enc}
        
    Returns:
        - rho_hom_eval: density matrix of the encrypted evaluated quantum state
        - c_list: a nested list of n empty list, where n is the dimension of the quantum state
    """    
    # density matrix representation of the encrypted state
    rho_hom_eval = pure_to_density(enc_psi)
    
    # dimension of the quantum state
    number_of_qubits = int(np.log2(enc_psi.shape[0]))
    
    # initializing a list which stores the `c` measured bits
    c_list = [[] for i in range(number_of_qubits)]
    
    for i in tcl_layers(circuits):
        counter = 0 
        
        # current_circuit iteratively builds up the quantum gate in each layer of the ciruit 
        current_circuit = np.array([1])
        for j in range(len(i)):
            if i[j] == 'I':
                current_circuit = np.kron(current_circuit, I)
                counter = counter + 1
                
            if i[j] == 'X':
                current_circuit = np.kron(current_circuit, X)
                counter = counter + 1

            if i[j] == 'Z':
                current_circuit = np.kron(current_circuit, Z)
                counter = counter + 1

            if i[j] == 'H':
                current_circuit = np.kron(current_circuit, H)
                counter = counter + 1

            if i[j] == 'P':
                current_circuit = np.kron(current_circuit, P)
                counter = counter + 1

            if i[j] == 'CNOT':
                current_circuit = np.kron(current_circuit, CNOT)
                counter = counter + 2
        
        rho_hom_eval = qgate_on_density(current_circuit,rho_hom_eval)
        
    circuit_dictionary = wire_dictionary(circuits) 
    return (rho_hom_eval, c_list)

def no_qubits(circuit):
    """
    Compute the number of qubits based on the circuit.
    
    Args: 
        - circuit: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        
    Returns: 
        - number_of_qubits: an integer corresponding to the number of qubits on which the circuit is acting upon.
    """
    number_of_qubits = 0
    first_layer = tcl_layers([circuit[0]])[0]
    for i in first_layer:
        if i == "CNOT":
            number_of_qubits = number_of_qubits + 2
        else:
            number_of_qubits = number_of_qubits + 1
    return(number_of_qubits)

def flatten_circuit(circuits):
    """
    Extract all the gates applied to each qubit in the same ordering of the qubits.
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        
    Returns: 
        - flattened_circuit: a nested list, where each sub-list contains the gates applied to each qubit 
        and each layer. If the CNOT gate is present, then CNOT(1) indicates the control qubit
        and CNOT(2) indicates the target qubit. 
    """
    number_of_qubits = no_qubits(circuits)
    individual_gates = tcl_layers(circuits)
    number_of_layers = len(individual_gates)
    flattened_circuit = [[] for _ in range(number_of_qubits)]
    for i in range(number_of_layers):
        counter = 0
        for j in range(number_of_qubits):
            if counter + 1 > number_of_qubits:
                break
            
            if individual_gates[i][j] != "CNOT":
                flattened_circuit[counter].append(individual_gates[i][j])
                counter = counter + 1
            else:
                flattened_circuit[counter].append("CNOT(1)")
                flattened_circuit[counter+1].append("CNOT(2)")
                counter = counter + 2

    return flattened_circuit

def number_of_tgates(circuit):
    """
    Count the number of T gates in a circuit.
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        
    Returns: 
        - t_gate_count: total number of T gates in a circuit as an integer
    """
    L = tcl_layers(circuit)
    t_gate_count = 0
    for i in L:
        for j in i:
            if j == "T":
                t_gate_count = t_gate_count + 1
    return t_gate_count

def wire_dictionary(circuits):
    """
    Construct a dictionary which contains information about the order of wires that act upon each qubit, 
    whether a T gate has been applied to a qubit during the evaluation of the EPR scheme or not, how many T gates
    will be acted upon each qubit and how many T gates have been applied during each iteration of the EPR evaluation 
    process. 
    This dictionary will provide a systematic way of keeping track of wires acting upon each qubit. 
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        
    Returns:
        - circuit_dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process. 
    """
    flat_circuit = flatten_circuit(circuits)
    number_of_qubits = len(flat_circuit)
    circuit_dictionary = {}
    for i in range(1, number_of_qubits+1):
        key = str(i)
        value = [i, False, number_of_tgates(flat_circuit[i-1]), 0]
        circuit_dictionary[key] = value
    return circuit_dictionary

def no_wires(circuits):
    """
    Count the total of number of wires required to run a circuit using the EPR evaluation scheme.
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        
    Returns:
        - number_of_wires: total number of wires, which is calculated by 
            number of qubits in the input message + 2 * number of T gates
    """
    number_of_qubits = no_qubits(circuits)
    t_gate_count = number_of_tgates(circuits)
    number_of_wires = number_of_qubits + 2 * t_gate_count
    return number_of_wires

def tgate_index(dictionary):
    """
    Construct a list consisting of the orderings of qubits on which a T gate is being applied.
    
    Args:
        - dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process.
            
    Returns: 
        - t_index_list: a list which records the orderings of qubits on which a T gate is being applied. 
    """
    t_index_list = []
    for key, value in dictionary.items():
        if dictionary[key][2] > 0:
            t_index_list.append(dictionary[key][0]-1)
    return t_index_list

def iterated_tensor_prod(M, n): # Compute M^{\otimes n}
    """
    Compute the matrix M^{\otimes n} with the convention that if n = 0, then the output is the integer 1. 
    
    Args:
        - M: A matrix
        - n: a non-negative integer
        
    Returns:
        - iterated_tensored_m: M^{\otimes n}
    """
    if n == 0:
        return 1
    iterated_tensored_m = M
    for i in range(n-1):
        iterated_tensored_m = np.kron(iterated_tensored_m, M)
    return iterated_tensored_m

def number_of_t_gates_applied(dictionary, qubit_order):
    """
    Count the total number of T gates applied to a given qubit during the evaluation procedure of the EPR scheme.
    
    Args: 
        - dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process.
        - qubit_order: the order of a qubit in a n-qubit system, an integer between 1 to n. 
        
    Returns: 
        - number_of_t_gates: an integer counting the number of T gates applied during
    """
    number_of_t_gates = 0
    for key in dictionary:
        if int(key)-1 < qubit_order:
            number_of_t_gates += dictionary[key][2]  # Access the third element of the list
    return number_of_t_gates

def controlled_not_constructor(control, target, number_of_qubits): 
    """
    This function applies the controlled-not operation on the target $t$ which is being controlled by $c$, 
    where $t < c$ or $c > t$ and the operation is being applied on a $n$-qubit system. 
    The operator matrix is defined as follows:

    if $t < c$: 

    $$CX_{c, t, n} = \left[ I^{\otimes(c-1)} \otimes | 0 \rangle \langle 0 | \otimes I^{\otimes(n-c)} \right] + 
    \left[ I^{\otimes(t-1)} \otimes X \otimes I^{\otimes(c-t-1)} \otimes | 1 \rangle \langle 1 | \otimes I^{\otimes (n-c)} \right]
    $$

    if $t > c$:

    $$
    CX_{c, t, n} = \left[ I^{\otimes(c-1)} \otimes | 0 \rangle \langle 0 | \otimes I^{\otimes(n-c)} \right]
    + \left[ I^{\otimes(c-1)} \otimes | 1 \rangle \langle 1 | \otimes I^{\otimes(t-c-1)} \otimes X \otimes I^{\otimes (n-t)}  \right]
    $$

    $I^{\otimes 0}$ is defined to be the real number $1$. 

    This function is used when applying the `T` gate during the EPR scheme.
    
    Args: 
        - control: the order of the control wire
        - target: the order of the target wire
        - number_of_qubits: number of qubits in the quantum system
        
    Returns:
        - controlled_not: the unitary operator which applies the X gate on 
        wire `target` if the `control` wire is 1.  
    """
    do_nothing_operator = np.kron(np.eye(2 ** (control - 1)), pure_to_density(zero_state()))
    do_nothing_operator = np.kron(do_nothing_operator, np.eye(2 ** (number_of_qubits - control)))
    if target < control: 
        flip_target_operator = np.eye(2 ** (target - 1))
        flip_target_operator = np.kron(flip_target_operator, X)
        flip_target_operator = np.kron(flip_target_operator, np.eye(2 ** (control - target - 1)))
        flip_target_operator = np.kron(flip_target_operator, pure_to_density(one_state()))
        flip_target_operator = np.kron(flip_target_operator, np.eye(2 ** (number_of_qubits - control)))
        
    else:
        flip_target_operator = np.eye(2 ** (control - 1))
        flip_target_operator = np.kron(flip_target_operator, pure_to_density(one_state()))
        flip_target_operator = np.kron(flip_target_operator, np.eye(2 ** (target - control - 1)))
        flip_target_operator = np.kron(flip_target_operator, X)
        flip_target_operator = np.kron(flip_target_operator, np.eye(2 ** (number_of_qubits - target)))

    controlled_not = do_nothing_operator + flip_target_operator
    return controlled_not


def epr_eval_no_fhe(circuits, enc_psi):    
    """
    Perform a circuit decomposed into Clifford + T gates on the encrypted quantum state
    homomorphically. 
    For more information, refer to the EPR scheme by Broadbent and Jeffery: https://arxiv.org/abs/1412.8766
    
    Args: 
        - circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
        - enc_psi: a vector representation of the encrypted pure state |psi>_{enc}

    Returns: 
        - rho_hom_eval: density matrix of the encrypted evaluated quantum state
        - c_list: a lists of bits resulting from the measurements of T gates. If no T gate was applied, 
        then it returns an empty list. 
    """    
    # Define variables for bookkeeping
    number_of_qubits = no_qubits(circuits)
    acting_wire_dictionary = wire_dictionary(circuits) 
    c_list = [[] for i in range(number_of_qubits)] 
    individual_gates = tcl_layers(circuits)
    number_of_wires = no_wires(circuits)
    t_gate_count = number_of_tgates(circuits)
    
    # if there are no T gates in the circuit, then apply the Clifford scheme evaluation procedure
    if t_gate_count == 0:
        return cl_eval_no_fhe(circuits, enc_psi)
    
    
    elif t_gate_count > 0:
        
        # append EPR pairs to the system
        enc_psi_eval = np.kron(enc_psi, iterated_tensor_prod(gen_epr(), t_gate_count))
        
    
    # density matrix of the encrypted state |psi>_{enc}
    rho_hom_eval = pure_to_density(enc_psi_eval)
    
    # apply SWAP gates
    for qubit_order in range(number_of_qubits):
        if acting_wire_dictionary[str(qubit_order+1)][2] == 0:
            continue 
        else: 
            # number of wires before the first T gate on the i^ht qubit
            n_w_b_t_f_t = number_of_qubits + number_of_t_gates_applied(acting_wire_dictionary, qubit_order) * 2 
            
            # number of T gates on the i^th qubit
            n_t_q_n = acting_wire_dictionary[str(qubit_order+1)][2] 
            
            starting_wire = n_w_b_t_f_t + 1
            current_t_wire = starting_wire
            for actingTwire in range(n_t_q_n * 2):
                current_t_wire = starting_wire + actingTwire
                for remaining_qubits in range(number_of_qubits-acting_wire_dictionary[str(qubit_order+1)][0]):
                    current_circuit = iterated_tensor_prod(I, current_t_wire-2)
                    current_circuit = np.kron(current_circuit, SWAP)
                    current_circuit = np.kron(current_circuit, 
                                              iterated_tensor_prod(I,number_of_wires - current_t_wire))
                    rho_hom_eval = qgate_on_density(current_circuit,rho_hom_eval)
                    
                    current_t_wire = current_t_wire - 1
                    
    # evaluation procedure begins
    for gate_order in range(len(individual_gates)):
        
        # we create a seperate variable to keep track of the order of the qubit
        # to handle the case when a CNOT is applied after which
        # the order of qubit will increase by 2. 
        qubit_order = 0
        
        for quasi_qubit_order in range(len(individual_gates[gate_order])):
            if acting_wire_dictionary[str(qubit_order+1)][3] == 0:
                acting_wire = acting_wire_dictionary[str(qubit_order+1)][0] + (
                    2*number_of_t_gates_applied(acting_wire_dictionary, qubit_order)
                )
            else: 
                acting_wire = acting_wire_dictionary[str(qubit_order+1)][0]
            
            current_gate = individual_gates[gate_order][quasi_qubit_order]
            
            # Clifford Gate
            if current_gate != "T":   
                if current_gate in ["I", "X", "Z"]:
                    if current_gate == "I":
                        current_circuit = iterated_tensor_prod(I, number_of_wires)


                    elif current_gate == "X":
                        current_circuit = iterated_tensor_prod(I, acting_wire-1)
                        current_circuit = np.kron(current_circuit, X)
                        current_circuit = np.kron(current_circuit, 
                                                  iterated_tensor_prod(I,number_of_wires - acting_wire))

                    else: 
                        current_circuit = iterated_tensor_prod(I, acting_wire-1)
                        current_circuit = np.kron(current_circuit, Z)
                        current_circuit = np.kron(current_circuit, 
                                                  iterated_tensor_prod(I,number_of_wires - acting_wire))

                    qubit_order += 1

                elif current_gate == "P":
                    current_circuit = iterated_tensor_prod(I, acting_wire-1)
                    current_circuit = np.kron(current_circuit, P)
                    current_circuit = np.kron(current_circuit, 
                                              iterated_tensor_prod(I,number_of_wires - acting_wire))

                    qubit_order += 1
                    

                elif current_gate == 'H':
                    current_circuit = iterated_tensor_prod(I, acting_wire-1)
                    current_circuit = np.kron(current_circuit, H)
                    current_circuit = np.kron(current_circuit, iterated_tensor_prod(I,number_of_wires - acting_wire))

                    qubit_order += 1
                
                # CNOT
                else:
                    control_wire = acting_wire
                    if acting_wire_dictionary[str(qubit_order+2)][3] == 0:
                        target_wire = acting_wire_dictionary[str(qubit_order+2)][0] + (
                            2*number_of_t_gates_applied(acting_wire_dictionary, qubit_order+1)
                        )
                    else: 
                        target_wire = acting_wire_dictionary[str(qubit_order+2)][0]
                
                    current_circuit = controlled_not_constructor(control_wire, target_wire, number_of_wires)

                    qubit_order += 2
                
                rho_hom_eval = qgate_on_density(current_circuit,rho_hom_eval)
    
            elif current_gate == "T":
            
                # keeping track of the control wire and target wire
                if acting_wire_dictionary[str(qubit_order+1)][1] == True: 
                    target_wire = acting_wire
                    control_wire = acting_wire + 2

                if acting_wire_dictionary[str(qubit_order+1)][1] == False:
                    acting_wire_dictionary[str(qubit_order+1)][1] = True
                    no_tgates_before_ith_qubit = number_of_t_gates_applied(acting_wire_dictionary, qubit_order)
                    target_wire = acting_wire
                    control_wire = (qubit_order) + 2*no_tgates_before_ith_qubit + 2

                
                # apply the T gate
                t_gate_operator = np.kron(iterated_tensor_prod(I, target_wire-1), T)
                t_on_rho_hom_eval = qgate_on_density(np.kron(
                t_gate_operator,iterated_tensor_prod(I, number_of_wires - target_wire)), rho_hom_eval)

                # apply the CNOT gate
                cnot_on_t_rho_hom_eval = qgate_on_density(controlled_not_constructor(control_wire, target_wire, number_of_wires), t_on_rho_hom_eval)
                
                # make measurement and obtain the post-measurement state and the classical bit c
                pms, c = post_measurement_state(cnot_on_t_rho_hom_eval, [target_wire-1], 
                                          measurement(partial_trace(cnot_on_t_rho_hom_eval, [target_wire-1])))

                # store the bit c in a list
                c_list[qubit_order].append(int(c))

                # perform the necessary updates
                acting_wire_dictionary[str(qubit_order+1)][0] = control_wire 
                acting_wire_dictionary[str(qubit_order+1)][3] = acting_wire_dictionary[str(qubit_order+1)][3] + 1
                rho_hom_eval = pms
                qubit_order += 1
     
    return (rho_hom_eval, c_list)



def number_of_tgates_dictionary(circuit_dictionary):
    """
    Count the total number of T gates in a circuit from the circuit dictionary.
    
    Args: 
        - circuit_dictionary: a dictonary object where keys indicate the ordering of qubits, starting from 1 as a string. 
        The values associated with each key is a list which contains the following information, in the same order:
            1) acting wire 
            2) a Boolean True/False value indicating whether a T gate has been applied during evaluation or not. 
            This value gets updated during the evaluation process.
            3) the total number of T gates that are going to be applied on the qubit as an integer. 
            4) the number of T gates applied at the end of the execution of each layer. This value gets updated
            during the evaluation process. 
        
    Returns:
        - number_of_t_gates: the total number of T gates in a circuit as an integer
    """
    number_of_t_gates = 0
    for key in circuit_dictionary:
        number_of_t_gates += circuit_dictionary[key][2]  # Access the third element of the list
    return number_of_t_gates

def epr_dec_no_fhe(rho_hom_eval, circuits, a, b, c_list):
    """
    Perform decryption of the encrypted evaluated cipherstate according to the EPR scheme without the 
    use of classical homomorphic encryption scheme.
    The key updates are computed according to the following table:
    
    ╒═════════╤═════════════════════════╤═════════════════════════════════════╕
    │ Gate    │ key                     │ updated key                         │
    ╞═════════╪═════════════════════════╪═════════════════════════════════════╡
    │ I, X, Z │ (a,b)                   │ (a,b)                               │
    ├─────────┼─────────────────────────┼─────────────────────────────────────┤
    │ H       │ (a,b)                   │ (b,a)                               │
    ├─────────┼─────────────────────────┼─────────────────────────────────────┤
    │ P       │ (a,b)                   │ (a, a + b)                          │
    ├─────────┼─────────────────────────┼─────────────────────────────────────┤
    │ CNOT    │ ((a_1, b_1),(a_2, b_2)) │ ((a_1, b_1 + b_2),(a_1 + a_2), b_2) │
    ├─────────┼─────────────────────────┼─────────────────────────────────────┤
    │ T       │ (a,b)                   │ (a+c, a+(a*c)+b+k)                  │
    ╘═════════╧═════════════════════════╧═════════════════════════════════════╛
    
    If the evaluated circuit does not consist of any T gates, then an application of the 
    quantum one-time pad is sufficient for decryption. 
    If there is at least one T gate in the circuit, then phase gates, conditional 
    on whether X Paulis were applied in the encryption of the quantum message, is required.
    After running a circuit consisting of a potential Phase gate and a Hadamard gate, 
    a measurement is performed to obtain the unknown `k` value, which then will be stored in a list.
    At the end, one last check is required before the performing the quantum one-time pad on  
    `a` and `b`. The evaluated decrypted density matrix of the input message is returned
    at the end.
    For more information, refer to the EPR scheme by Broadbent and Jeffery: https://arxiv.org/abs/1412.8766. 
    
    
    Args: 
         - rho_hom_eval: density matrix of the encrypted evaluated quantum state
         - description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit. 
         - a: a list of bits needed for the X-Pauli gates of the quantum one-time pad
         - b: a list of bits needed for the Z-Pauli gates of the quantum one-time pad
         - c_list: a list of bits needed for updating the keys
            
    Returns: 
         - rho_dec_eval: evaluated decrypted density matrix of the input message 
    """
    # Define variables for bookkeeping
    number_of_wires = int(np.log2(rho_hom_eval.shape[0]))
    t_gate_count = number_of_tgates(circuits)
    number_of_qubits = len(c_list)
    gates_by_layers = tcl_layers(circuits)
    k_list = [[] for i in range(number_of_qubits)]
    flat_circuit = flatten_circuit(circuits)
    acting_wire_dictionary = wire_dictionary(circuits)
    t_index_list = tgate_index(acting_wire_dictionary)
    
    a_copy = copy.deepcopy(a)
    b_copy = copy.deepcopy(b)
    rho_dec_eval = rho_hom_eval
    
    for gate_order in range(len(gates_by_layers)):
        qubit_order = 0
        for quasi_qubit_order in range(len(gates_by_layers[gate_order])):
            current_gate = gates_by_layers[gate_order][quasi_qubit_order]
        
            if current_gate in ["I", "X", "Z"]:
                a_copy[qubit_order] = a_copy[qubit_order]
                b_copy[qubit_order] = b_copy[qubit_order]
                qubit_order += 1
                
            elif current_gate == "P":
                a_copy[qubit_order] = a_copy[qubit_order]
                b_copy[qubit_order] = a_copy[qubit_order] ^ b_copy[qubit_order]
                qubit_order += 1
                
            elif current_gate == "H":
                temp = a_copy[qubit_order]
                a_copy[qubit_order] = b_copy[qubit_order]
                b_copy[qubit_order] = temp
                qubit_order += 1
                
            elif current_gate == "CNOT":
                a_copy[qubit_order] = a_copy[qubit_order]
                b_copy[qubit_order] = b_copy[qubit_order] ^ b_copy[qubit_order+1]
                a_copy[qubit_order+1] = a_copy[qubit_order] ^ a_copy[qubit_order+1]
                b_copy[qubit_order+1] = b_copy[qubit_order+1]
                qubit_order += 2
                
            elif current_gate == "T": # T-Gate
                if acting_wire_dictionary[str(qubit_order+1)][3] == 0:
                    acting_wire_dictionary[str(qubit_order+1)][1] = True 
                    acting_wire_dictionary[str(qubit_order+1)][3] = acting_wire_dictionary[str(qubit_order+1)][3] + 1
                    no_t_gates_before_ith_qubit = number_of_t_gates_applied(acting_wire_dictionary, qubit_order)
                    acting_wire = qubit_order + 2*no_t_gates_before_ith_qubit + 3
                    temp_list = acting_wire_dictionary[str(qubit_order+1)]
                    temp_list.append(acting_wire)
                    acting_wire_dictionary[str(qubit_order+1)] = temp_list
                    
                else:
                    acting_wire = acting_wire_dictionary[str(qubit_order+1)][4] + 2
                    acting_wire_dictionary[str(qubit_order+1)][3] = acting_wire_dictionary[str(qubit_order+1)][3] + 1
                    acting_wire_dictionary[str(qubit_order+1)][4] = acting_wire
                    
                t_gate_count_on_ith_qubit = acting_wire_dictionary[str(qubit_order+1)][3] - 1
                p_exponent_value = a_copy[qubit_order]
                
                p_operator = np.kron(iterated_tensor_prod(I,acting_wire-1), 
                                     matrix_power(P, p_exponent_value))
                p_operator = np.kron(p_operator, iterated_tensor_prod(I,number_of_wires-acting_wire))

                phase_on_rho_dec_eval = qgate_on_density(p_operator, rho_dec_eval)
                
                h_operator = np.kron(iterated_tensor_prod(I,acting_wire-1), H)
                h_operator = np.kron(h_operator, iterated_tensor_prod(I,number_of_wires-acting_wire))
                h_on_phase_rho_dec_eval = qgate_on_density(h_operator, phase_on_rho_dec_eval)
                
                
                pms, k = post_measurement_state(h_on_phase_rho_dec_eval, 
                                               [acting_wire - 1],  # -1 to account for acting_wire starts from 1
                                               # python indexing starts from 0
                                               measurement(partial_trace(h_on_phase_rho_dec_eval, [acting_wire - 1])))
                
                k_list[qubit_order].append(int(k))
                rho_dec_eval = pms
                
                b_copy[qubit_order] = a_copy[qubit_order] ^ (a_copy[qubit_order] * c_list[qubit_order][t_gate_count_on_ith_qubit]) ^ b_copy[qubit_order] ^ int(k)
                a_copy[qubit_order] = a_copy[qubit_order] ^ c_list[qubit_order][t_gate_count_on_ith_qubit]
                
                
                qubit_order += 1
                
                
                
    acting_wires = []  
    if t_gate_count > 0:
        for i in range(number_of_qubits):
            if len(acting_wire_dictionary[str(i+1)]) == 5:
                acting_wires.append(acting_wire_dictionary[str(i+1)][4]-2)
            if len(acting_wire_dictionary[str(i+1)]) == 4:
                acting_wires.append(i + 2 * number_of_t_gates_applied(acting_wire_dictionary, i))

        rho_dec_eval = qotp_dec_density(partial_trace(rho_dec_eval, acting_wires), 
                                     a_copy, 
                                     b_copy)

    elif t_gate_count == 0:
        rho_dec_eval = qotp_dec_density(rho_dec_eval, 
                                     a_copy, 
                                     b_copy)
    
    
    return rho_dec_eval


def epr_qhe_no_fhe(psi, a, b, circuits):
    """
    Homomorphically apply the quantum circuit provided by `circuits` on the quantum pure state
    |psi>, which is encrypted with bits `a` and `b` using the quantum one-time pad according to the EPR scheme.
    In this modified version, classical fully homomorphic encryption is not used and instead the classical key 
    updates are done by the client during the decryption. 
    For more information refer to the Clifford scheme by Broadbent and Jeffery: https://arxiv.org/abs/1412.8766
    
    Args:
        psi: a vector representation of the pure state |psi>
        a: list of (randomly generated) classical bits
        b: list of (randomly generated) classical bits
        circuits: description of the quantum circuit as a list of strigs. Each element of the list represents one layer
        of the circuit.
        
    Returns:
        rho_dec_eval: evaluated decrypted density matrix of the input message 
    """ 
    psi_enc = qotp_enc(psi, a, b)
    rho_hom_eval, c_list = epr_eval_no_fhe(circuits,psi_enc)
    rho_dec_eval = epr_dec_no_fhe(rho_hom_eval, circuits, a, b, c_list)
    
    return rho_dec_eval